# Terrain Creation with USGS 3DEP Data

This notebook demonstrates downloading USGS 1m LiDAR terrain and creating HEC-RAS terrain HDF files.

**Primary Workflow**:
1. Extract example project to get project extents
2. Download USGS 3DEP 1m LiDAR tiles directly from AWS S3
3. Mosaic tiles into single TIFF
4. Create terrain HDF from TIFF using RasProcess.exe
5. Register terrain in RasMap for use in HEC-RAS

**Prerequisites**:
- HEC-RAS 6.6+ installed (for RasProcess.exe CreateTerrain command)
- Python packages: rasterio, geopandas, h5py

**Data Source**: [USGS 3D Elevation Program (3DEP)](https://www.usgs.gov/3d-elevation-program)
- 1-meter resolution LiDAR (highest quality, widespread coverage)
- Data accessed directly from AWS S3 (public bucket, no credentials required)
- Automatic tile discovery and downloading

## Setup and Imports

In [1]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
# Set USE_LOCAL_SOURCE based on your setup:
#   True  = Use local source code (for developers editing ras-commander)  
#   False = Use pip-installed package (for users)
# =============================================================================

USE_LOCAL_SOURCE = True  # <-- TOGGLE THIS

# -----------------------------------------------------------------------------
if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)  # Parent of examples/ = repo root
    if local_path not in sys.path:
        sys.path.insert(0, local_path)  # Insert at position 0 = highest priority
    print(f"LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("PIP PACKAGE MODE: Loading installed ras-commander")

# Import ras-commander
from ras_commander import RasExamples, RasMap, init_ras_project, ras
from ras_commander.terrain import RasTerrain, Usgs3depAws

# Verify which version loaded
import ras_commander
print(f"Loaded: {ras_commander.__file__}")

LOCAL SOURCE MODE: Loading from c:\GH\ras-commander/ras_commander
Loaded: c:\GH\ras-commander\ras_commander\__init__.py


In [2]:
# Additional imports for terrain operations
from pathlib import Path
import h5py
import numpy as np
import subprocess
import shutil
import warnings

# Check for required packages
try:
    import rasterio
    from rasterio.crs import CRS
    RASTERIO_AVAILABLE = True
    print("rasterio available")
except ImportError:
    RASTERIO_AVAILABLE = False
    print("rasterio not available - install with: pip install rasterio")
    print("Cannot proceed without rasterio")

try:
    import geopandas as gpd
    GEOPANDAS_AVAILABLE = True
    print("geopandas available")
except ImportError:
    GEOPANDAS_AVAILABLE = False
    print("geopandas not available - install with: pip install geopandas")

try:
    from osgeo import gdal
    GDAL_AVAILABLE = True
    print("gdal available")
except ImportError:
    GDAL_AVAILABLE = False
    print("gdal (osgeo) not available - VRT creation will use HEC-RAS GDAL")

rasterio available
geopandas available
gdal (osgeo) not available - VRT creation will use HEC-RAS GDAL


## Parameters

Configure these values to customize the notebook for your environment.

In [3]:
# =============================================================================
# PARAMETERS - Edit these to customize the notebook
# =============================================================================

# Project Configuration
PROJECT_NAME = "BaldEagleCrkMulti2D"  # Example project with 2D terrain
RAS_VERSION = "6.6"  # HEC-RAS version

# Terrain Download Configuration
TARGET_RESOLUTION = 1  # meters (1m LiDAR is highest quality available)
NEW_TERRAIN_NAME = "Terrain_3DEP"  # Name for new terrain we'll create
VERTICAL_UNITS = "Feet"  # Feet or Meters

# HEC-RAS Installation Path (auto-detected)
HECRAS_PATH = Path(f"C:/Program Files (x86)/HEC/HEC-RAS/{RAS_VERSION}")
RAS_PROCESS_EXE = HECRAS_PATH / "RasProcess.exe"

print(f"Project: {PROJECT_NAME}")
print(f"HEC-RAS Version: {RAS_VERSION}")
print(f"Target Resolution: {TARGET_RESOLUTION}m")
print(f"RasProcess.exe exists: {RAS_PROCESS_EXE.exists()}")

Project: BaldEagleCrkMulti2D
HEC-RAS Version: 6.6
Target Resolution: 1m
RasProcess.exe exists: True


## Extract Example Project

We'll extract an example project to get the project extents for downloading USGS terrain.

In [4]:
# Extract the example project with a suffix for this notebook
project_path = RasExamples.extract_project(PROJECT_NAME, suffix="920")
print(f"Project extracted to: {project_path}")

# Initialize the project
init_ras_project(project_path, RAS_VERSION)
print(f"Project initialized: {ras.project_name}")

# Create terrain folder if needed
terrain_folder = project_path / "Terrain"
terrain_folder.mkdir(exist_ok=True)
print(f"Terrain folder: {terrain_folder}")

2026-01-13 10:30:53 - ras_commander.RasExamples - INFO - Found zip file: C:\GH\ras-commander\examples\Example_Projects_6_6.zip
2026-01-13 10:30:53 - ras_commander.RasExamples - INFO - Loading project data from CSV...
2026-01-13 10:30:53 - ras_commander.RasExamples - INFO - Loaded 68 projects from CSV.
2026-01-13 10:30:53 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----
2026-01-13 10:30:53 - ras_commander.RasExamples - INFO - Extracting project 'BaldEagleCrkMulti2D' as 'BaldEagleCrkMulti2D_920'
2026-01-13 10:30:53 - ras_commander.RasExamples - INFO - Folder 'BaldEagleCrkMulti2D_920' already exists. Deleting existing folder...
2026-01-13 10:30:54 - ras_commander.RasExamples - INFO - Existing folder 'BaldEagleCrkMulti2D_920' has been deleted.
2026-01-13 10:30:56 - ras_commander.RasExamples - INFO - Successfully extracted project 'BaldEagleCrkMulti2D' to C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_920
2026-01-13 10:30:56 - ras_commander.

Project extracted to: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_920
Project initialized: BaldEagleDamBrk
Terrain folder: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_920\Terrain


## Get Project Extents

Extract the bounding box from the project's existing terrain or geometry to know where to download USGS data.

In [5]:
# Find existing terrain to get extents
existing_tiffs = list(terrain_folder.glob("*.tif"))
existing_hdfs = list(terrain_folder.glob("*.hdf"))

print(f"Found {len(existing_tiffs)} TIFF files")
print(f"Found {len(existing_hdfs)} HDF files")

# Get bounds from first TIFF if available
if existing_tiffs and RASTERIO_AVAILABLE:
    source_tiff = existing_tiffs[0]
    print(f"\nUsing {source_tiff.name} to determine project extents:")
    
    with rasterio.open(source_tiff) as src:
        bounds = src.bounds
        crs = src.crs
        print(f"  Bounds (native CRS): {bounds}")
        print(f"  CRS: {crs}")
        print(f"  Resolution: {src.res[0]:.2f} x {src.res[1]:.2f}")
        print(f"  Size: {src.width} x {src.height} pixels")
        
        # Store for later use
        PROJECT_BOUNDS = bounds
        PROJECT_CRS = crs
else:
    print("No existing TIFF found - will need to specify bounds manually")
    PROJECT_BOUNDS = None
    PROJECT_CRS = None

Found 2 TIFF files
Found 1 HDF files

Using Terrain50.baldeagledem.tif to determine project extents:
  Bounds (native CRS): BoundingBox(left=1834327.1955903, bottom=162918.80517492996, right=2149835.693237871, top=414872.9473435675)
  CRS: EPSG:2271
  Resolution: 36.50 x 36.50
  Size: 8643 x 6902 pixels


In [6]:
# Convert bounds to WGS84 (lat/lon) for USGS 3DEP
if PROJECT_BOUNDS and RASTERIO_AVAILABLE:
    from pyproj import Transformer
    from shapely.geometry import box
    
    # Create transformer from project CRS to WGS84
    transformer = Transformer.from_crs(PROJECT_CRS, "EPSG:4326", always_xy=True)
    
    # Transform corners
    minx, miny = transformer.transform(PROJECT_BOUNDS.left, PROJECT_BOUNDS.bottom)
    maxx, maxy = transformer.transform(PROJECT_BOUNDS.right, PROJECT_BOUNDS.top)
    
    # Calculate area
    full_width_deg = maxx - minx
    full_height_deg = maxy - miny
    full_area_sq_km = full_width_deg * 111 * full_height_deg * 85  # Rough approximation
    
    print(f"Full terrain extent:")
    print(f"  Bounds (WGS84): ({minx:.6f}, {miny:.6f}) to ({maxx:.6f}, {maxy:.6f})")
    print(f"  Approx area: {full_area_sq_km:.1f} sq km")
    
    # Use full extent for download (Usgs3depAws will download only intersecting tiles)
    DOWNLOAD_BBOX = (minx, miny, maxx, maxy)
    
    print(f"\nDownload bounding box (WGS84):")
    print(f"  West:  {DOWNLOAD_BBOX[0]:.6f}")
    print(f"  South: {DOWNLOAD_BBOX[1]:.6f}")
    print(f"  East:  {DOWNLOAD_BBOX[2]:.6f}")
    print(f"  North: {DOWNLOAD_BBOX[3]:.6f}")
else:
    print("Cannot determine download bounds without existing terrain")
    DOWNLOAD_BBOX = None

Full terrain extent:
  Bounds (WGS84): (-78.233233, 40.612784) to (-77.089997, 41.303441)
  Approx area: 7449.7 sq km

Download bounding box (WGS84):
  West:  -78.233233
  South: 40.612784
  East:  -77.089997
  North: 41.303441


## Download USGS 3DEP Terrain

Download 1m LiDAR terrain tiles directly from USGS AWS S3 using `Usgs3depAws`.

**How it works**:
1. Downloads USGS tile index (~15 MB GeoPackage, one-time)
2. Finds projects covering your area
3. Downloads file list from each project
4. Checks which tiles intersect your area
5. Downloads only intersecting tiles (typically 2-10 tiles)

**Tile sizes**: ~200-300 MB per tile (10km x 10km at 1m resolution)

In [7]:
# Find USGS 3DEP projects covering this area
if DOWNLOAD_BBOX and GEOPANDAS_AVAILABLE:
    print("Finding USGS 3DEP 1m projects for this area...")
    print(f"This will download a ~15 MB tile index (one-time, cached).")
    
    try:
        projects = Usgs3depAws.find_tiles_for_bbox(
            bbox=DOWNLOAD_BBOX,
            resolution=TARGET_RESOLUTION,
            cache_folder="cache/usgs3dep"
        )
        
        print(f"\nFound {len(projects)} project(s) with {TARGET_RESOLUTION}m data:")
        for idx, proj in projects.iterrows():
            print(f"  - {proj['project']} (published: {proj['pub_date']})")
        
        USGS_PROJECTS = projects
    except Exception as e:
        print(f"Could not find projects: {e}")
        USGS_PROJECTS = None
else:
    print("Cannot find projects - missing bbox or geopandas")
    USGS_PROJECTS = None

2026-01-13 10:30:56 - ras_commander.terrain.Usgs3depAws - INFO - Using cached 1m tile index: cache\usgs3dep\FESM_1m.gpkg


Finding USGS 3DEP 1m projects for this area...
This will download a ~15 MB tile index (one-time, cached).


2026-01-13 10:31:06 - ras_commander.terrain.Usgs3depAws - INFO -   Loaded 870 tiles
2026-01-13 10:31:06 - ras_commander.terrain.Usgs3depAws - INFO - Found 3 projects intersecting bbox



Found 3 project(s) with 1m data:
  - PA_Northcentral_2019_B19 (published: 2022-05-02 00:00:00)
  - PA_South_Central_2017_D17 (published: 2019-04-16 00:00:00)
  - PA_South_Central_2017_D17 (published: 2019-04-11 00:00:00)


In [8]:
# Download 1m DEM tiles from USGS 3DEP (AWS S3)
if DOWNLOAD_BBOX and USGS_PROJECTS is not None and len(USGS_PROJECTS) > 0:
    print(f"Downloading {TARGET_RESOLUTION}m DEM tiles for project area...")
    print(f"Note: This checks each tile in project(s) for intersection (~10 min for large projects)")
    print(f"      Tiles are cached - subsequent runs will be much faster.")
    print()
    
    try:
        # Download intersecting tiles
        tile_folder = terrain_folder / "usgs_tiles"
        downloaded_tiles = Usgs3depAws.download_tiles(
            bbox=DOWNLOAD_BBOX,
            resolution=TARGET_RESOLUTION,
            output_folder=tile_folder,
            cache_folder="cache/usgs3dep"
        )
        
        print(f"\nDownloaded {len(downloaded_tiles)} tiles:")
        total_size_mb = 0
        for tile in downloaded_tiles:
            size_mb = tile.stat().st_size / 1024 / 1024
            total_size_mb += size_mb
            print(f"  {tile.name} ({size_mb:.2f} MB)")
        
        print(f"\nTotal size: {total_size_mb:.2f} MB")
        
        DOWNLOADED_TILES = downloaded_tiles
    except Exception as e:
        print(f"Download failed: {e}")
        import traceback
        traceback.print_exc()
        DOWNLOADED_TILES = None
else:
    print("Cannot download - missing bbox or no projects found")
    DOWNLOADED_TILES = None

2026-01-13 10:31:06 - ras_commander.terrain.Usgs3depAws - INFO - Using cached 1m tile index: cache\usgs3dep\FESM_1m.gpkg


Note: This checks each tile in project(s) for intersection (~10 min for large projects)
      Tiles are cached - subsequent runs will be much faster.



2026-01-13 10:31:15 - ras_commander.terrain.Usgs3depAws - INFO -   Loaded 870 tiles
2026-01-13 10:31:15 - ras_commander.terrain.Usgs3depAws - INFO - Found 3 projects intersecting bbox
2026-01-13 10:31:15 - ras_commander.terrain.Usgs3depAws - INFO - Found 3 projects intersecting bbox
2026-01-13 10:31:15 - ras_commander.terrain.Usgs3depAws - INFO -   Selecting most recent project (year 2019)
2026-01-13 10:31:15 - ras_commander.terrain.Usgs3depAws - INFO -     Selected: PA_Northcentral_2019_B19
2026-01-13 10:31:15 - ras_commander.terrain.Usgs3depAws - INFO -     Skipping 2 older project(s):
2026-01-13 10:31:15 - ras_commander.terrain.Usgs3depAws - INFO -       - PA_South_Central_2017_D17 (year 2017)
2026-01-13 10:31:15 - ras_commander.terrain.Usgs3depAws - INFO -       - PA_South_Central_2017_D17 (year 2017)
2026-01-13 10:31:15 - ras_commander.terrain.Usgs3depAws - INFO - 
Processing project: PA_Northcentral_2019_B19
2026-01-13 10:31:15 - ras_commander.terrain.Usgs3depAws - INFO - Fetchin


Downloaded 98 tiles:
  USGS_1M_17_x76y456_PA_Northcentral_2019_B19.tif (312.02 MB)
  USGS_1M_17_x76y455_PA_Northcentral_2019_B19.tif (308.78 MB)
  USGS_1M_17_x76y453_PA_Northcentral_2019_B19.tif (299.71 MB)
  USGS_1M_17_x76y457_PA_Northcentral_2019_B19.tif (293.74 MB)
  USGS_1M_17_x76y454_PA_Northcentral_2019_B19.tif (320.38 MB)
  USGS_1M_17_x76y451_PA_Northcentral_2019_B19.tif (317.89 MB)
  USGS_1M_17_x76y452_PA_Northcentral_2019_B19.tif (295.65 MB)
  USGS_1M_17_x76y450_PA_Northcentral_2019_B19.tif (303.93 MB)
  USGS_1M_17_x75y455_PA_Northcentral_2019_B19.tif (295.42 MB)
  USGS_1M_17_x75y456_PA_Northcentral_2019_B19.tif (292.75 MB)
  USGS_1M_17_x75y457_PA_Northcentral_2019_B19.tif (311.43 MB)
  USGS_1M_17_x75y454_PA_Northcentral_2019_B19.tif (295.85 MB)
  USGS_1M_17_x75y453_PA_Northcentral_2019_B19.tif (311.08 MB)
  USGS_1M_17_x75y452_PA_Northcentral_2019_B19.tif (294.59 MB)
  USGS_1M_17_x75y451_PA_Northcentral_2019_B19.tif (310.71 MB)
  USGS_1M_17_x74y457_PA_Northcentral_2019_B19.ti

In [9]:
# Create mosaic from downloaded tiles
# Uses GDAL VRT if available, otherwise falls back to rasterio merge
if DOWNLOADED_TILES and len(DOWNLOADED_TILES) > 0:
    print(f"Creating mosaic from {len(DOWNLOADED_TILES)} tiles...")
    output_tiff = terrain_folder / f"{NEW_TERRAIN_NAME}.tif"
    
    try:
        if GDAL_AVAILABLE:
            # Method 1: GDAL VRT approach (faster for large datasets)
            print("Using GDAL VRT approach...")
            vrt_path = terrain_folder / f"{NEW_TERRAIN_NAME}.vrt"
            
            vrt = Usgs3depAws.create_vrt(
                tile_files=DOWNLOADED_TILES,
                output_vrt=vrt_path
            )
            print(f"Created VRT mosaic: {vrt.name}")
            
            # Convert VRT to single TIFF using RasTerrain
            print(f"\nConverting VRT to TIFF...")
            print(f"Note: This may take several minutes for large mosaics")
            
            final_tiff = RasTerrain.vrt_to_tiff(
                vrt_path=vrt,
                output_path=output_tiff,
                compression="LZW",
                create_overviews=True,
                hecras_version=RAS_VERSION
            )
        else:
            # Method 2: Rasterio merge (no GDAL dependency)
            print("Using rasterio merge approach (GDAL not available)...")
            print("Note: This may take several minutes for large mosaics")
            from rasterio.merge import merge
            
            # Open all tiles and merge
            src_files = [rasterio.open(tile) for tile in DOWNLOADED_TILES]
            mosaic, out_transform = merge(src_files)
            
            # Get metadata from first file
            out_meta = src_files[0].meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": mosaic.shape[1],
                "width": mosaic.shape[2],
                "transform": out_transform,
                "compress": "lzw"
            })
            
            # Close source files
            for src in src_files:
                src.close()
            
            # Write merged TIFF
            with rasterio.open(output_tiff, "w", **out_meta) as dest:
                dest.write(mosaic)
            
            final_tiff = output_tiff
            print(f"Merged {len(DOWNLOADED_TILES)} tiles using rasterio")
        
        print(f"\nSaved: {final_tiff}")
        print(f"  Size: {final_tiff.stat().st_size / 1024 / 1024:.2f} MB")
        
        # Get info about the TIFF
        with rasterio.open(final_tiff) as src:
            print(f"  Shape: {src.height} x {src.width} pixels")
            print(f"  CRS: {src.crs}")
            print(f"  Resolution: {src.res[0]:.2f} x {src.res[1]:.2f}")
            print(f"  Bounds: {src.bounds}")
        
        DOWNLOADED_TIFF = final_tiff
    except Exception as e:
        print(f"Mosaic/conversion failed: {e}")
        import traceback
        traceback.print_exc()
        DOWNLOADED_TIFF = None
else:
    print("No tiles to mosaic")
    DOWNLOADED_TIFF = None

Creating mosaic from 98 tiles...
Using rasterio merge approach (GDAL not available)...
Note: This may take several minutes for large mosaics
Mosaic/conversion failed: Unable to allocate 181. GiB for an array with shape (1, 90012, 540012) and data type float32


Traceback (most recent call last):
  File "C:\Users\billk_clb\AppData\Local\Temp\ipykernel_165176\1063916843.py", line 38, in <module>
    mosaic, out_transform = merge(src_files)
                            ~~~~~^^^^^^^^^^^
  File "c:\Users\billk_clb\anaconda3\envs\rascmdr_local\Lib\site-packages\rasterio\merge.py", line 427, in merge
    dest = np.zeros((output_count, chunk.height, chunk.width), dtype=dt)
numpy._core._exceptions._ArrayMemoryError: Unable to allocate 181. GiB for an array with shape (1, 90012, 540012) and data type float32


## Create Terrain HDF

Use `RasTerrain.create_terrain_hdf()` to create HEC-RAS terrain from the downloaded TIFF.

This uses `RasProcess.exe CreateTerrain` from HEC-RAS 6.6+ which:
- Creates multi-resolution pyramid levels (7 levels, 0-6)
- Enables TIN stitching for multi-source terrain
- Optimizes tile-based storage for HEC-RAS rendering

In [10]:
# Find or create projection file
prj_files = list(terrain_folder.glob("*.prj"))
print(f"Found {len(prj_files)} projection files")

if prj_files:
    projection_prj = prj_files[0]
    print(f"Using existing: {projection_prj.name}")
elif DOWNLOADED_TIFF and DOWNLOADED_TIFF.exists():
    # Generate PRJ from the TIFF
    projection_prj = terrain_folder / "Projection.prj"
    print(f"Generating PRJ from {DOWNLOADED_TIFF.name}...")
    
    RasTerrain._generate_prj_from_raster(
        DOWNLOADED_TIFF,
        projection_prj
    )
    print(f"Created: {projection_prj.name}")
else:
    print("No projection file available - terrain creation will fail")
    projection_prj = None

Found 1 projection files
Using existing: Projection.prj


In [11]:
# Create terrain HDF using RasTerrain class
if DOWNLOADED_TIFF and DOWNLOADED_TIFF.exists() and projection_prj and projection_prj.exists():
    output_hdf = terrain_folder / f"{NEW_TERRAIN_NAME}.hdf"
    
    print(f"Creating terrain HDF:")
    print(f"  Input TIFF: {DOWNLOADED_TIFF.name}")
    print(f"  Output HDF: {output_hdf.name}")
    print(f"  Projection: {projection_prj.name}")
    print(f"  Units: {VERTICAL_UNITS}")
    print()
    
    try:
        result_hdf = RasTerrain.create_terrain_hdf(
            input_rasters=[DOWNLOADED_TIFF],
            output_hdf=output_hdf,
            projection_prj=projection_prj,
            units=VERTICAL_UNITS,
            stitch=True,
            hecras_version=RAS_VERSION
        )
        
        print(f"\nSUCCESS! Terrain HDF created")
        print(f"  File: {result_hdf}")
        print(f"  Size: {result_hdf.stat().st_size / 1024 / 1024:.2f} MB")
        
        NEW_TERRAIN_HDF = result_hdf
    except Exception as e:
        print(f"\nERROR: Terrain creation failed")
        print(f"  {e}")
        print("\nTroubleshooting:")
        print("  1. Ensure HEC-RAS 6.6+ is installed")
        print(f"  2. Check RasProcess.exe exists: {RAS_PROCESS_EXE.exists()}")
        print("  3. Verify TIFF file is valid (open in QGIS/ArcGIS)")
        NEW_TERRAIN_HDF = None
else:
    print("Cannot create terrain - missing input files")
    if not DOWNLOADED_TIFF or not DOWNLOADED_TIFF.exists():
        print("  Missing: Downloaded TIFF file")
    if not projection_prj or not projection_prj.exists():
        print("  Missing: Projection PRJ file")
    NEW_TERRAIN_HDF = None

Cannot create terrain - missing input files
  Missing: Downloaded TIFF file


### Fallback: Use Existing Terrain

If USGS download or terrain creation failed, use the existing terrain from the example project.

In [12]:
# Fallback to existing terrain if new terrain wasn't created
if NEW_TERRAIN_HDF is None or not NEW_TERRAIN_HDF.exists():
    existing_hdfs = list(terrain_folder.glob("*.hdf"))
    
    if existing_hdfs:
        NEW_TERRAIN_HDF = existing_hdfs[0]
        print(f"Using existing terrain: {NEW_TERRAIN_HDF.name}")
        print(f"  Size: {NEW_TERRAIN_HDF.stat().st_size / 1024 / 1024:.2f} MB")
        # Update name for RasMap registration
        NEW_TERRAIN_NAME = NEW_TERRAIN_HDF.stem
    else:
        print("No terrain HDF available")
else:
    print(f"Using newly created terrain: {NEW_TERRAIN_HDF.name}")

Using existing terrain: Terrain50.hdf
  Size: 4.69 MB


## Register Terrain in RasMap

After creating a terrain HDF, register it in the project's `.rasmap` file so HEC-RAS can use it.

In [13]:
# Find the .rasmap file
rasmap_files = list(project_path.glob("*.rasmap"))
print(f"Found {len(rasmap_files)} .rasmap files:")
for rm in rasmap_files:
    print(f"  {rm.name} ({rm.stat().st_size / 1024:.1f} KB)")

Found 1 .rasmap files:
  BaldEagleDamBrk.rasmap (43.9 KB)


In [14]:
# Inspect existing terrain configuration in .rasmap
import xml.etree.ElementTree as ET

if rasmap_files:
    rasmap_file = rasmap_files[0]
    tree = ET.parse(rasmap_file)
    root = tree.getroot()
    
    # Find Terrains section
    terrains = root.find(".//Terrains")
    if terrains is not None:
        print(f"Existing Terrains section in {rasmap_file.name}:")
        
        # List terrain layers
        layers = terrains.findall("Layer")
        print(f"\n  Terrain layers ({len(layers)}):")
        for layer in layers:
            print(f"    - Name: {layer.get('Name')}")
            print(f"      Filename: {layer.get('Filename')}")
    else:
        print("No Terrains section found in .rasmap file")

Existing Terrains section in BaldEagleDamBrk.rasmap:

  Terrain layers (1):
    - Name: Terrain50
      Filename: .\Terrain\Terrain50.hdf


In [15]:
# Add the new terrain layer to the .rasmap file
if rasmap_files and NEW_TERRAIN_HDF and NEW_TERRAIN_HDF.exists():
    rasmap_file = rasmap_files[0]
    
    print(f"Adding terrain layer to {rasmap_file.name}:")
    print(f"  Terrain HDF: {NEW_TERRAIN_HDF.name}")
    print(f"  Layer name: {NEW_TERRAIN_NAME}")
    
    try:
        RasMap.add_terrain_layer(
            terrain_hdf=NEW_TERRAIN_HDF,
            rasmap_path=rasmap_file,
            layer_name=NEW_TERRAIN_NAME,
            projection_prj=projection_prj if projection_prj and projection_prj.exists() else None
        )
        print(f"\nTerrain layer added successfully!")
    except Exception as e:
        print(f"\nError adding terrain layer: {e}")
elif not NEW_TERRAIN_HDF or not NEW_TERRAIN_HDF.exists():
    print(f"No terrain HDF to add")
else:
    print("No .rasmap files found in project")

2026-01-13 10:59:41 - ras_commander.RasMap - INFO - Replaced existing terrain layer: Terrain50
2026-01-13 10:59:41 - ras_commander.RasMap - INFO - Updated projection reference: .\Terrain\Projection.prj
2026-01-13 10:59:41 - ras_commander.RasMap - INFO - Added terrain layer 'Terrain50' to .rasmap file: .\Terrain\Terrain50.hdf


Adding terrain layer to BaldEagleDamBrk.rasmap:
  Terrain HDF: Terrain50.hdf
  Layer name: Terrain50

Terrain layer added successfully!


In [16]:
# Verify the terrain layer was added
if rasmap_files:
    rasmap_file = rasmap_files[0]
    tree = ET.parse(rasmap_file)
    root = tree.getroot()
    
    terrains = root.find(".//Terrains")
    if terrains is not None:
        layers = terrains.findall("Layer")
        print(f"Updated Terrains section ({len(layers)} layers):")
        for layer in layers:
            name = layer.get('Name')
            marker = " <-- NEW" if name == NEW_TERRAIN_NAME else ""
            print(f"  - {name}: {layer.get('Filename')}{marker}")

Updated Terrains section (1 layers):
  - Terrain50: .\Terrain\Terrain50.hdf <-- NEW


## Summary

This notebook demonstrated the complete terrain creation workflow:

1. **Extract Project** - Get project extents from existing terrain

2. **Download USGS 3DEP** - Used `py3dep` to download high-resolution terrain:
   - Automatic resolution selection (1m/3m/10m/30m)
   - Reprojection to project CRS
   - Conversion to project vertical units (feet/meters)

3. **Create Terrain HDF** - Used `RasTerrain.create_terrain_hdf()`:
   - Multi-resolution pyramid levels (7 levels)
   - TIN stitching for seamless terrain
   - Optimized for HEC-RAS rendering

4. **Register in RasMap** - Added terrain to project configuration

### Next Steps

- Open project in HEC-RAS and verify terrain displays correctly
- For bathymetric merging, see `RasTerrain` multi-source terrain support
- For terrain modification, see geometry HDF documentation

---

# Appendix: Terrain HDF Structure Analysis

The following cells analyze the internal structure of HEC-RAS terrain HDF files for reference.

In [17]:
def print_hdf_structure(hdf_file, prefix="", max_depth=3, current_depth=0):
    """Recursively print HDF5 file structure with dataset info."""
    for key in hdf_file.keys():
        item = hdf_file[key]
        if isinstance(item, h5py.Group):
            print(f"{prefix}{key}/")
            if current_depth < max_depth:
                print_hdf_structure(item, prefix + "  ", max_depth, current_depth + 1)
        else:
            # Dataset - show shape and dtype
            shape_str = f"shape={item.shape}" if hasattr(item, 'shape') else ""
            dtype_str = f"dtype={item.dtype}" if hasattr(item, 'dtype') else ""
            print(f"{prefix}{key} ({shape_str}, {dtype_str})")


# Find terrain HDF files to analyze
terrain_hdfs = list(terrain_folder.glob("*.hdf"))
print(f"Found {len(terrain_hdfs)} terrain HDF files:")
for hdf in terrain_hdfs:
    print(f"  {hdf.name} ({hdf.stat().st_size / 1024 / 1024:.2f} MB)")

Found 1 terrain HDF files:
  Terrain50.hdf (4.69 MB)


In [18]:
# Open and explore the first terrain HDF
if terrain_hdfs:
    terrain_hdf = terrain_hdfs[0]
    print(f"\nExploring: {terrain_hdf.name}")
    print("=" * 60)
    
    with h5py.File(terrain_hdf, 'r') as hdf:
        print_hdf_structure(hdf, max_depth=2)
else:
    print("No terrain HDF files found in Terrain folder")


Exploring: Terrain50.hdf
Terrain/
  Stitch TIN Points (shape=(72592, 4), dtype=float64)
  Stitch TIN Triangles (shape=(72301, 3), dtype=int32)
  Stitches (shape=(103000, 7), dtype=float64)
  Terrain50.baldeagledem/
    0/
    1/
    2/
    3/
    4/
    5/
    6/
  Terrain50.dtm_20ft/
    0/
    1/
    2/
    3/
    4/
    5/


In [19]:
# Inspect the stitching data structure
if terrain_hdfs:
    with h5py.File(terrain_hdfs[0], 'r') as hdf:
        terrain_group = hdf.get('Terrain')
        
        if terrain_group:
            # Check for stitch data
            if 'Stitch TIN Points' in terrain_group:
                stitch_points = terrain_group['Stitch TIN Points'][:]
                print(f"Stitch TIN Points:")
                print(f"  Shape: {stitch_points.shape}")
                print(f"  Columns: X, Y, Z, M (material/priority)")
                print(f"  Sample (first 3 points):")
                for i, pt in enumerate(stitch_points[:3]):
                    print(f"    [{i}] X={pt[0]:.2f}, Y={pt[1]:.2f}, Z={pt[2]:.2f}, M={pt[3]:.0f}")
            
            if 'Stitch TIN Triangles' in terrain_group:
                stitch_tris = terrain_group['Stitch TIN Triangles'][:]
                print(f"\nStitch TIN Triangles:")
                print(f"  Shape: {stitch_tris.shape}")
                print(f"  Columns: p0, p1, p2 (vertex indices)")
                
            # List source terrain layers
            print(f"\nSource terrain layers:")
            for key in terrain_group.keys():
                if key not in ['Stitch TIN Points', 'Stitch TIN Triangles', 'Stitches']:
                    print(f"  {key}")
        else:
            print("No Terrain group found in HDF")

Stitch TIN Points:
  Shape: (72592, 4)
  Columns: X, Y, Z, M (material/priority)
  Sample (first 3 points):
    [0] X=2075224.54, Y=371231.14, Z=796.59, M=20
    [1] X=2075224.54, Y=371251.14, Z=803.22, M=20
    [2] X=2075244.54, Y=371251.14, Z=803.09, M=20

Stitch TIN Triangles:
  Shape: (72301, 3)
  Columns: p0, p1, p2 (vertex indices)

Source terrain layers:
  Terrain50.baldeagledem
  Terrain50.dtm_20ft


In [20]:
# Inspect pyramid levels in a source terrain layer
if terrain_hdfs:
    with h5py.File(terrain_hdfs[0], 'r') as hdf:
        terrain_group = hdf.get('Terrain')
        
        if terrain_group:
            # Find first source layer (not stitch data)
            source_layers = [k for k in terrain_group.keys() 
                            if k not in ['Stitch TIN Points', 'Stitch TIN Triangles', 'Stitches']]
            
            if source_layers:
                layer_name = source_layers[0]
                layer = terrain_group[layer_name]
                print(f"Source layer: {layer_name}")
                print(f"\nPyramid levels (resolution hierarchy):")
                
                # List pyramid levels (0-6 typically)
                for level_key in sorted([k for k in layer.keys() if k.isdigit()]):
                    level = layer[level_key]
                    print(f"\n  Level {level_key}:")
                    for dataset_key in level.keys():
                        ds = level[dataset_key]
                        print(f"    {dataset_key}: shape={ds.shape}, dtype={ds.dtype}")

Source layer: Terrain50.baldeagledem

Pyramid levels (resolution hierarchy):

  Level 0:
    Mask: shape=(918, 65536), dtype=uint8
    Min-Max: shape=(918, 3), dtype=float32
    Perimeter: shape=(918, 513), dtype=float32

  Level 1:
    Mask: shape=(238, 65536), dtype=uint8
    Min-Max: shape=(238, 3), dtype=float32
    Perimeter: shape=(238, 513), dtype=float32

  Level 2:
    Mask: shape=(63, 65536), dtype=uint8
    Min-Max: shape=(63, 3), dtype=float32
    Perimeter: shape=(63, 513), dtype=float32

  Level 3:
    Mask: shape=(20, 65536), dtype=uint8
    Min-Max: shape=(20, 3), dtype=float32
    Perimeter: shape=(20, 513), dtype=float32

  Level 4:
    Mask: shape=(6, 65536), dtype=uint8
    Min-Max: shape=(6, 3), dtype=float32
    Perimeter: shape=(6, 513), dtype=float32

  Level 5:
    Mask: shape=(2, 65536), dtype=uint8
    Min-Max: shape=(2, 3), dtype=float32
    Perimeter: shape=(2, 513), dtype=float32

  Level 6:
    Mask: shape=(1, 65536), dtype=uint8
    Min-Max: shape=(1, 3)

## GUI Verification (Optional)

To verify the terrain displays correctly in HEC-RAS:

1. Open HEC-RAS and load the project
2. Open RAS Mapper (View → RAS Mapper)
3. Expand the Terrains section
4. Check the new terrain layer to display it
5. Verify elevations and coverage look correct

## Cleanup

In [21]:
# Optional: Clean up extracted project
CLEANUP_PROJECT = False  # Set to True to remove extracted project

if CLEANUP_PROJECT and project_path.exists():
    print(f"Cleaning up: {project_path}")
    shutil.rmtree(project_path, ignore_errors=True)
    print("Project folder removed")
else:
    print(f"Project preserved at: {project_path}")
    print("Set CLEANUP_PROJECT = True to remove on next run")

Project preserved at: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D_920
Set CLEANUP_PROJECT = True to remove on next run
